## Audio Analysis on White Speech Audio data

In [8]:
import pandas as pd
import numpy as np
np.random.seed(1001)

import os
import shutil

import IPython
import matplotlib
import matplotlib.pyplot as plt

import seaborn as sns
from tqdm import tqdm_notebook
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score


%matplotlib inline
matplotlib.style.use('ggplot')

In [21]:
import librosa
import requests

In [5]:
df = pd.read_csv('/Users/will/Documents/GitHub/Bias-in-ASR/white_audio.csv')
df

,Unnamed: 0,row_idx,id,speaker_id,chapter_id,audio_url,text
0,0,0,374-180298-0000,374,180298,https://datasets-server.huggingface.co/cached-...,CHAPTER SIXTEEN I MIGHT HAVE TOLD YOU OF THE B...
1,1,1,374-180298-0001,374,180298,https://datasets-server.huggingface.co/cached-...,MARGUERITE TO BE UNABLE TO LIVE APART FROM ME ...
2,2,2,374-180298-0002,374,180298,https://datasets-server.huggingface.co/cached-...,I WISHED ABOVE ALL NOT TO LEAVE MYSELF TIME TO...
3,3,3,374-180298-0003,374,180298,https://datasets-server.huggingface.co/cached-...,ASSUMED ALL AT ONCE AN APPEARANCE OF NOISE AND...
4,4,4,374-180298-0004,374,180298,https://datasets-server.huggingface.co/cached-...,NOTHING IS SO EXPENSIVE AS THEIR CAPRICES FLOW...
...,...,...,...,...,...,...,...
695,95,695,5750-35690-0008,5750,35690,https://datasets-server.huggingface.co/cached-...,AND BUT LITTLE AT THIS DAY EXCEPT IT BE FOR A ...
696,96,696,5750-35690-0009,5750,35690,https://datasets-server.huggingface.co/cached-...,MANY THAT DID ILL UNDER PHYSICIANS HANDS HAVE ...
697,97,697,5750-35690-0010,5750,35690,https://datasets-server.huggingface.co/cached-...,LEFT TO GOD AND NATURE AND THEMSELVES
698,98,698,5750-35690-0011,5750,35690,https://datasets-server.huggingface.co/cached-...,JURIDICIS MEDICIS FISCO FAS VIVERE RAPTO TIS A...


#### Data Cleaning

In [6]:
df.drop(columns=['Unnamed: 0','row_idx'])

,id,speaker_id,chapter_id,audio_url,text
0,374-180298-0000,374,180298,https://datasets-server.huggingface.co/cached-...,CHAPTER SIXTEEN I MIGHT HAVE TOLD YOU OF THE B...
1,374-180298-0001,374,180298,https://datasets-server.huggingface.co/cached-...,MARGUERITE TO BE UNABLE TO LIVE APART FROM ME ...
2,374-180298-0002,374,180298,https://datasets-server.huggingface.co/cached-...,I WISHED ABOVE ALL NOT TO LEAVE MYSELF TIME TO...
3,374-180298-0003,374,180298,https://datasets-server.huggingface.co/cached-...,ASSUMED ALL AT ONCE AN APPEARANCE OF NOISE AND...
4,374-180298-0004,374,180298,https://datasets-server.huggingface.co/cached-...,NOTHING IS SO EXPENSIVE AS THEIR CAPRICES FLOW...
...,...,...,...,...,...
695,5750-35690-0008,5750,35690,https://datasets-server.huggingface.co/cached-...,AND BUT LITTLE AT THIS DAY EXCEPT IT BE FOR A ...
696,5750-35690-0009,5750,35690,https://datasets-server.huggingface.co/cached-...,MANY THAT DID ILL UNDER PHYSICIANS HANDS HAVE ...
697,5750-35690-0010,5750,35690,https://datasets-server.huggingface.co/cached-...,LEFT TO GOD AND NATURE AND THEMSELVES
698,5750-35690-0011,5750,35690,https://datasets-server.huggingface.co/cached-...,JURIDICIS MEDICIS FISCO FAS VIVERE RAPTO TIS A...


#### Data Analysis

In [15]:
def extract_mfccs(file_path, n_mfcc=13):
    y, sr = librosa.load(file_path)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    # Returning the mean of each MFCC across time, simpler representation for analysis
    return np.mean(mfccs, axis=1)

In [16]:
def extract_pitch(file_path):
    y, sr = librosa.load(file_path)
    pitches, magnitudes = librosa.core.piptrack(y=y, sr=sr)
    dominant_pitches = [np.max(pitch) for pitch in pitches.transpose()]
    # Returning the average pitch for simplicity
    return np.mean(dominant_pitches)

In [17]:
def extract_duration(file_path):
    y, sr = librosa.load(file_path)
    return librosa.get_duration(y=y, sr=sr)

In [20]:
df.loc[0]['audio_url']

'https://datasets-server.huggingface.co/cached-assets/librispeech_asr/--/8aeb8cac5ad165fc5574d4e84218154a8f4eca7b/--/all/train.clean.100/0/audio/audio.wav?Expires=1711399917&Signature=aQITsDpVGzhC6V6xfOHg6I6-jDPmdtBLlO4leMHd1RoW6xfEYFTQUte7B8-FNgrHDAIarRSZ0RT28AYFZXMEizRfeJtLDBHoj6vlQ4dXQSftEQXh74qNfuEqvdvi67v5A3BYP87KwKN-MGV9oOnld40ySUkd49~Cqti9axdvb74m5jYRcNh~v7~XYewIaOBsxbSTupaiUyBW30tUOUHMVnN6Cifnh5jXhSPA4Or2YRDK18hcUwyOZuTqlQjIbLmB0saRfNXmtkwzH01jx-QMgkH2tqXL6bzoKv-k1xzqkxdSh60R17O5PEi5R3u-BI990YPc9w-Yamjx-a4YGwGxDg__&Key-Pair-Id=K3EI6M078Z3AC3'

In [22]:
def download_audio(url, save_path):
    response = requests.get(url)
    if response.status_code == 200:
        with open(save_path, 'wb') as audio_file:
            audio_file.write(response.content)
        print(f"Downloaded: {save_path}")
    else:
        print(f"Failed to download: {url}")

In [24]:
save_directory = '/Users/will/Documents/GitHub/Bias-in-ASR/training-audios'

df['audio_url'].apply(lambda url: download_audio(url, os.path.join(save_directory, url.split('/')[-1])))

Failed to download: https://datasets-server.huggingface.co/cached-assets/librispeech_asr/--/8aeb8cac5ad165fc5574d4e84218154a8f4eca7b/--/all/train.clean.100/0/audio/audio.wav?Expires=1711399917&Signature=aQITsDpVGzhC6V6xfOHg6I6-jDPmdtBLlO4leMHd1RoW6xfEYFTQUte7B8-FNgrHDAIarRSZ0RT28AYFZXMEizRfeJtLDBHoj6vlQ4dXQSftEQXh74qNfuEqvdvi67v5A3BYP87KwKN-MGV9oOnld40ySUkd49~Cqti9axdvb74m5jYRcNh~v7~XYewIaOBsxbSTupaiUyBW30tUOUHMVnN6Cifnh5jXhSPA4Or2YRDK18hcUwyOZuTqlQjIbLmB0saRfNXmtkwzH01jx-QMgkH2tqXL6bzoKv-k1xzqkxdSh60R17O5PEi5R3u-BI990YPc9w-Yamjx-a4YGwGxDg__&Key-Pair-Id=K3EI6M078Z3AC3
Failed to download: https://datasets-server.huggingface.co/cached-assets/librispeech_asr/--/8aeb8cac5ad165fc5574d4e84218154a8f4eca7b/--/all/train.clean.100/1/audio/audio.wav?Expires=1711399917&Signature=VXnC2hdasIyqy1TU6g7WgwRRi0zwwYFeO992ze46GZPWimIUvOIpLl7~cCXx-KCdkrjLc4QcK~cAQw50jg~QAQtn9-iRDTkHmVZA~NQi6JkWVadB1FSkNrhdzuBIt46aSB6d6CmEI0l3qo0RDyX3SOUkvnRI5U7pW0nhN8EuMU10FHzx8WZ8M58IriuYRll53C3OGDnsfmYHD5AsBzxW3b7bHw6BHV

Failed to download: https://datasets-server.huggingface.co/cached-assets/librispeech_asr/--/8aeb8cac5ad165fc5574d4e84218154a8f4eca7b/--/all/train.clean.100/15/audio/audio.wav?Expires=1711399917&Signature=XjiyVwMDxh6SYUBxbsYWiXVELTUk9ner~XjLEaig6P7Im6IJQ1fx~ramaNw8deOsWGgdT0JmRyxtpWDcBzIfFhrIK5RDHGXYPx1acen2ncj2MZAwRHB0Ux~mdKoa6IXMQpE7uHX5EJ7I0i84zfieh94YlT0gFAtVjwXSNUcIvQmy41U6lNNvGwZ5nX0Ok7y1npohB86ix78W7prYKI-eS00o687CK5ZgQ83zUX9enWtE2zkk1~1yJQ7bxNZ~MQF-zF04KmHHssi3pM9IqiGwrYjgA4Eq5zGQ1b9zVpelCPLEOneQGBQP1gdaHZGukA7oDZMZIgxGxKOL9MpQTuHAgw__&Key-Pair-Id=K3EI6M078Z3AC3
Failed to download: https://datasets-server.huggingface.co/cached-assets/librispeech_asr/--/8aeb8cac5ad165fc5574d4e84218154a8f4eca7b/--/all/train.clean.100/16/audio/audio.wav?Expires=1711399917&Signature=swmUPkjZXLLogzh5ED2FHi-sGY8SRlqcYiU1E9tDqNdY0Z5TN-Zm3Hl-uDHkZRdupl5yLq~OBMgW~QJACcvyXAsErL17unP0UczO0F7Q9selDBidF-66itvc2krP26-jfoa3kEFqmpfn7VqYnSbxH1ZHYzM9BaRi0TB1NfQEbOCSkdJsaIKL9kvCTIxwG5JoQG12MYcavnCjZpDdw1G75sLU20tB

Failed to download: https://datasets-server.huggingface.co/cached-assets/librispeech_asr/--/8aeb8cac5ad165fc5574d4e84218154a8f4eca7b/--/all/train.clean.100/31/audio/audio.wav?Expires=1711399917&Signature=sx0oUlUnvotkTUG7PCvKtcbg9oLgz7jXreyU5ns6OjYlWo~FbmXD7Eigg3VzUnrotzY-zN6pV-BKdoKbL9EzrweQrsAHo4OPy14rPzRzRwNDBBfOSBjSgl97jtcPBpJGUw42CL2dzVoSHnCOdUx4C90i16KpCuBkPfvCnzBoNMjfII2~ACq-Yq5atanCpg2~KeLIlUW8qBqgxHEq3A~cs9GQ2TwzQKUtqwwMgJD6ra2x1Yu7mL3iKDgL-Mr0Hw8To2KaL-8RF0-0SVF0oRFR6nJQtdAAY5G-Eqnd4i97HWnYwaaAlnT159TgyRTDa9z2Wef0lU0MFqA2LI9pjhat-A__&Key-Pair-Id=K3EI6M078Z3AC3
Failed to download: https://datasets-server.huggingface.co/cached-assets/librispeech_asr/--/8aeb8cac5ad165fc5574d4e84218154a8f4eca7b/--/all/train.clean.100/32/audio/audio.wav?Expires=1711399917&Signature=HlSMaE25qme07PEuSb0c0KwuhHxaNoMsvH1rUMd2mERoynexkDNK3ud6OJpYBzVfv8i-9XMjgF4crC9pVKQHd6FNITBvyYriKEZTNk2iLcVi70mFPIuVWOldGvCgCnybDGg5RSwDm1JJSLDQkACMmSj6kveTCAOp6cZcQE25hm~4RBfLMuMohmdTmSorGLUhO5ahs9f4pur9AzanNprVQVPOgLDI

Failed to download: https://datasets-server.huggingface.co/cached-assets/librispeech_asr/--/8aeb8cac5ad165fc5574d4e84218154a8f4eca7b/--/all/train.clean.100/47/audio/audio.wav?Expires=1711399917&Signature=0VB06hggswS3eP2HqoxJO9amdUwglvQU4Z9gG4vLNbUUdTcyCdeIufcn5NvxfKNz1nAQ9G9gZFUm9a5c2vtLzyz~6uEdzUkL7hbDWC69SO54g2iTpHUVBjlV~qterppNwSoOrl35JAjKepp~VuiIX3xMavH1Y8~CwThapW4uC56031KUScPD3Z2unlT370NO0pCgvKiJrSC3dwwDR61rg-OK4nZI41tLDvE1~1ZG086fuJaMYXfj~yUNL-s9yeNBJ~VIArJSrhxdIdH4V6yUIi7gUirMR2lCnzgCuj1h37czrPdl-FxV99u1pmo79mI35x9UyyaX42AAn2k0DylDVQ__&Key-Pair-Id=K3EI6M078Z3AC3
Failed to download: https://datasets-server.huggingface.co/cached-assets/librispeech_asr/--/8aeb8cac5ad165fc5574d4e84218154a8f4eca7b/--/all/train.clean.100/48/audio/audio.wav?Expires=1711399917&Signature=gjbYb~MEsuRUxSbi1l8FQ2DMycHIUjs~3k2sGhLjC2HIW6d-LPgobyuTwgEV~GHzvk7wj7uFy0x0oGi9sxtPZiW4Av-PbQG6OtoUm7HWIDQH2cXnxbks~GoGYQV2Y0NhU10~xk08XsZ3IQwkcdIgUp5N2mNOKEeibdz90VB2JA8u3N48eY9l6a7hQXaqF4izXX6SdYssBB3y47uD4diVxxREVPyl

Failed to download: https://datasets-server.huggingface.co/cached-assets/librispeech_asr/--/8aeb8cac5ad165fc5574d4e84218154a8f4eca7b/--/all/train.clean.100/63/audio/audio.wav?Expires=1711399917&Signature=v9MyEMKGrS9NA59fkqFeE4yKfRxE0Bci3PXxIB4laYdpOxoYOMaPST0UCbO4kX7utCgCR7v1uFuHjCdk5qFpuy0tKo9e3-PUAOdaOGEwxEBToeL~jO0tAI5kzDSPKAYkV9JIfQIDspFTCxkMLOSyjYCcoVrBM~G8csjrVpvyBf6G3zLMPCuKuVXe5~LuPLA26llxgzMZvwcu4ennTLzIgfqhO8PgKq-OaFhoZXUzGcc7LpVGs-1KEoeYR-ifOkqW-rWHUnsDI3EwiG8GsuDMLnaVGoo5Rjve48jVseV4BF83k0qslEy0PaqckufQJfPM096Hnvs0H2LC9gZoO3~o1w__&Key-Pair-Id=K3EI6M078Z3AC3
Failed to download: https://datasets-server.huggingface.co/cached-assets/librispeech_asr/--/8aeb8cac5ad165fc5574d4e84218154a8f4eca7b/--/all/train.clean.100/64/audio/audio.wav?Expires=1711399917&Signature=sJSW~hMlbWSdWNKHXMR-QTZ7kM71ngFJMINj40e4aADdxXUILXJ0PElpT0ZT-Jq7a2vPoVKzS6r6lMSFUYXCvIR1zBy8kz-Ncr9DPpocBggS2Qw-~a9f30B8s9NfR2UhNqEjLB4uawlv1kaRLT0TIOS-q7YtJf-6xcGMCRgIsnEovahFLK0iBa9EW3rHmwx4DsW9kxmsa0tpSE0wCyMIfl3ciiVy

...

Failed to download: https://datasets-server.huggingface.co/cached-assets/librispeech_asr/--/8aeb8cac5ad165fc5574d4e84218154a8f4eca7b/--/all/train.clean.100/217/audio/audio.wav?Expires=1711400296&Signature=JQBz9dAMX0eh5HVfJ8SaFC28PqmHnG6Yic5TgFs2fIFOyfiT8ZguPlQMkH3~Ge5KhhSir~m5fngXw9oDT0oYqDM9WDx83enJDsNp5Uyn7dWlIO546kJ7mroef5nkP7IjKuOKotIx7xAkBK9j82FdC9UoDyBu8PzSLOXdpPlgGbnSvdia9L9IftdiwC94ogdgtgCqbZaqayJOatxC7WeAj8rn9erH3AzkZ~vl70TvEbYSemIr91N-OzQcQY6b0I~f0F4fPuQxsWHLwdAPRGhZekXIV5J8m7turJSgMX81cwy7W4LAAiGx-EigfT0W5Nzl2emOeunvLeOfJJtKlPoMaw__&Key-Pair-Id=K3EI6M078Z3AC3
Failed to download: https://datasets-server.huggingface.co/cached-assets/librispeech_asr/--/8aeb8cac5ad165fc5574d4e84218154a8f4eca7b/--/all/train.clean.100/218/audio/audio.wav?Expires=1711400296&Signature=Xj8vk3SXqxKNjjT9juv3p3y5TbCc-noPsnR8smnUvJoi9gKjIrJhp27sLSh0rxUWl~YsxFeT6U0bgqlhld3xs8D6K5AteOzZZ67R5fDd1ePRoMz42-vvLN7wUwWeYibUSX8E6sL9DTwUPStataM-fqiNapXQLkthErpfzIxn2XEHVsBITnHPnEgi0KjfCCMx3uhcyWNu9MRMoB3My8U41lF2en

Failed to download: https://datasets-server.huggingface.co/cached-assets/librispeech_asr/--/8aeb8cac5ad165fc5574d4e84218154a8f4eca7b/--/all/train.clean.100/232/audio/audio.wav?Expires=1711400296&Signature=NqErmMaOiL8uEs1h01aJahJvvrAL0IeoOuXUraMcdj4Z69Afz3EHGLZ44wrGDU6p9DYyo-PD6XfKw3w6WIHTUdipeqr2McLfwfbWwpm5-UmAx2I0a2TVcJKwlxcBDNw~edg9sPX235KypRLo1zvOK2wx7QPZ7YiUgmCXSnov6XcB96-EGwkyJiNElEuKAImxXkKfTBx2Vqb~5XHDhh9mhq-JqLfmptm1FGDxKGyLiFGGFZ4Xv6ThGfJBOnfLVWjnutDChPjZGgOMNcX33-haj3a8SGM0fWzYXyDNLLkCUAg31ZGWUhmuHc6NfIEEMZAm1j0N5PQQSWKfYTnhtn7y6A__&Key-Pair-Id=K3EI6M078Z3AC3
Failed to download: https://datasets-server.huggingface.co/cached-assets/librispeech_asr/--/8aeb8cac5ad165fc5574d4e84218154a8f4eca7b/--/all/train.clean.100/233/audio/audio.wav?Expires=1711400296&Signature=XRG7aBKgePgSEyGH-2w32yeVzmA8uMkl~xsqqE0IQ4DZj4D9ATHQNoKOoQinXt9MKqvogfq-Ij0OiDpI98CfzRCzYaztB4QZM6tUgBvzoHXG3MfakLVbQIhORKsfWyFefu8K4ZT2STiX533HvjhxbnYlKq-PEjP47LRrFCCTFN9dVa4OMBu7jLFjLDAwDzUVZWA2q0w~-6yepqSzOyQW9o~54c

Failed to download: https://datasets-server.huggingface.co/cached-assets/librispeech_asr/--/8aeb8cac5ad165fc5574d4e84218154a8f4eca7b/--/all/train.clean.100/248/audio/audio.wav?Expires=1711400296&Signature=quIc8B8455oeyCtU8k8GmE28EVRBNVa6wcrTxowD8lbtBg2HD5wj9y75WWrmFdYU7SjHP31wQ108VLFod7hexXklp4k~jm~hpuWgyKN5XstmMTebnLacFHVlQ1DMMRgpUyQH~Bg~SDbhhkSlLzQaMIXJA07wBcDhz7tDlSWuWkdVBlNJMBg6mlQC5Z6MHTMGINB1zb-fDeqcjWE1RLqHeRuSL-UKQa-FraufsXiTHdHwY6IpBTOkvb5ipn-hKUwg9erUcgwefdcw1IDkEx7tIlFyCYVGM2N3A3OHOeVglxTCJPfWMFs3NijMnqaJ7NyJobBvQ0RPAVym5PVn1eG9iw__&Key-Pair-Id=K3EI6M078Z3AC3
Failed to download: https://datasets-server.huggingface.co/cached-assets/librispeech_asr/--/8aeb8cac5ad165fc5574d4e84218154a8f4eca7b/--/all/train.clean.100/249/audio/audio.wav?Expires=1711400296&Signature=DKOlJ-3xfDKd4QTZwUbExxpD4~lrJqMCpqvSu5knyGu-9MK2OlBJq7hY2YfK924yVUh74hO-0vGv1JuLrHxPTqTu1RgCp~0JKrJzaJYGbD1HstCyitRTv2i9-QG9oHq6THgBUpdR8jfyCF~FqGFlktdLZdsk8Hk7wfp4Cvjg1b239PKBbY-NJUipsl0Fe50h8nEw9oyt1sEJpx84opZN5gjHdU

Failed to download: https://datasets-server.huggingface.co/cached-assets/librispeech_asr/--/8aeb8cac5ad165fc5574d4e84218154a8f4eca7b/--/all/train.clean.100/264/audio/audio.wav?Expires=1711400296&Signature=ZR84LUGG62XICsv7OlUJWDcVZfLtAwZfS-QWweJ9UwCnRonTdv6yyCOHzITHSg7BxAOa8-268hZf5-lGvUiBY1L2v6f2OJT5W2r1Auun4k2kS4I3n4iQuYKxcdezPRCoN1Q6KJAOaBHr1c8KzsGzQzNmquXNw-ml0Ls~2YoPlx6H88KbkDSu~AFWjgppOXynJA~uMIXRtyttscYTo1QlqmIL4ivdSRK4so0BXDBnp5RF39-7vdKGG69rXYjIyQW-uIaJn8cjd3d373PSrA8I0M85HBkDVaeDb7rN4UvN~4RAC0xRMOPtkCzkvyK87FE5JUEAioGSdx-CRLAHyXmXGQ__&Key-Pair-Id=K3EI6M078Z3AC3
Failed to download: https://datasets-server.huggingface.co/cached-assets/librispeech_asr/--/8aeb8cac5ad165fc5574d4e84218154a8f4eca7b/--/all/train.clean.100/265/audio/audio.wav?Expires=1711400296&Signature=qXEeccZYXaBRbQPEzAnAshm3OcjfnBHb2SIW0b46QS9wI2xWUn6c~JHBr05tSmP69PSa3DCQyxNHTP-EMevxYVB0KC83i9K-Re6aAPFiiR6kyJXOBnOtfu580NNvyi4AlhEWb20fUcxBC9Cfr2RRVBSWAMWIkNOAyg8eBWV7b6WfRUVx2SLvPtuosvcKWInUrFGtgo3yfHDlTjwEtXSbY1WNLV

KeyboardInterrupt: 